<a href="https://colab.research.google.com/github/ankitstar01/WFH/blob/master/Copy_of_reocgnizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To run this colab, press the "Runtime" button in the menu tab and then press the "Run all" button.

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recognize Flowers using Transfer Learning

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/community/en/flowers_tf_lite.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/community/en/flowers_tf_lite.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
import tensorflow as tf
assert tf.__version__.startswith('2')
import numpy as np
from functools import partial
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import os
import zipfile
from PIL import ImageOps,Image
import cv2
import time
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPool2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import seaborn as sns
import shutil
from sklearn.metrics import confusion_matrix,classification_report
NAME='Activity-cnn-64x2-{}'.format(int(time.time()))

tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
training_data=[]
cat=['running','drinking']

IMG_SIZE=224

#download

def download_file(url,file_name):
    !mkdir data
    data_dir = 'data/'
    file_name = file_name+'.'+os.path.split(url)[-1].split('.')[-1]
    file_path = os.path.join(data_dir, file_name)
    if not os.path.exists(file_path):
          # Download
        file_url = urllib.request.urlopen(url)
        with open(file_path, 'wb') as output:
            output.write(file_url.read())
            if file_name.split('.')[1]=='zip':
            # Extract
               with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(data_dir)
        print('downloaded')
    return file_path,file_name
def unzip(path,outputPath):
    if path.split('.')[1]=='zip' or path.split('.')[1]=='rar':
      # Extract
      with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(outputPath)

In [0]:
# #downaload things
shutil.rmtree('/content/test')
!mkdir test
# datadir,name=download_file('http://vision.stanford.edu/Datasets/Stanford40.zip','Activity')
unzip('/content/test.zip','./')
# shutil.rmtree('/content/temp1')
# shutil.rmtree('/content/temp1/party')
# shutil.rmtree('/content/temp1/shopping')
# shutil.rmtree('/content/temp1/travelling')
# shutil.rmtree('/content/temp1/Shopping_data')
# shutil.rmtree('/content/temp1/Travelling_Data')
# !mkdir temp1
# shutil.copy('/content/drive/My Drive/Colab Notebooks/datasets/Travelling_Data.zip','/content/temp1/')
# unzip('/content/temp1/Shopping_data.zip')

In [0]:
# shutil.rmtree('/content/temp1')
!mkdir temp1
downdir = 'data/JPEGImages/'
act=['brushing_teeth','cooking','drinking','gardening','party','phoning','riding_a_bike','riding_a_horse','running','shopping','taking_photos','travelling']
# act=['Bicycling_NEW','Exercise_NEW','Sitting_NEW','Standing_NEW','Yoga_NEW']
!mkdir temp1/running
!mkdir temp1/phoning
!mkdir temp1/gardening
!mkdir temp1/cooking
!mkdir temp1/drinking
!mkdir temp1/taking_photos
!mkdir temp1/brushing_teeth
!mkdir temp1/riding_a_bike
!mkdir temp1/riding_a_horse
updir='tmp/'
for file in os.listdir(downdir):
  label=file[:-8]
  if label in act:
    file_path='/content/data/JPEGImages/'+file
    im=Image.open(file_path)
    im_mirror = ImageOps.mirror(im)
    im_mirror.save('/content/temp1/{}/flip_{}'.format(label,file)) 
    shutil.copy(file_path,'/content/temp1/{}'.format(label)) 



# !mkdir temp1/Standing_NEW
# !mkdir temp1/Sitting_NEW
# !mkdir temp1/Yoga_NEW
# !mkdir temp1/Exercise_NEW
# !mkdir temp1/Bicycling_NEW
# base_dir='/content/drive/My Drive/Colab Notebooks/datasets/Data_set_ankit/'
# for folder in os.listdir(base_dir):
#   c=0
#   folder_path='/content/drive/My Drive/Colab Notebooks/datasets/Data_set_ankit/{}'.format(folder)
#   for file in os.listdir(folder_path):
#     input_path=folder_path+'/{}'.format(file)
#     print(input_path)
#     c=c+1
#     if c>600:
#       print('break')
#       break
#     if folder in act:
#       _=shutil.copy(input_path,'/content/temp1/{}'.format(folder)) 

unzip('/content/drive/My Drive/Colab Notebooks/datasets/party.zip','/content/temp1')
for file in os.listdir('/content/temp1/party/'):
  im=Image.open('/content/temp1/party/{}'.format(file))
  im_mirror=ImageOps.mirror(im)
  im_mirror.save('/content/temp1/party/flip_{}'.format(file))
print('party : ',len(os.listdir('/content/temp1/party/')))

# !mkdir temp1/shopping
# unzip('/content/drive/My Drive/Colab Notebooks/datasets/Shopping_data.zip','/content/temp1')
# os.rename('/content/temp1/Shopping_data','/content/temp1/shopping')
# for file in os.listdir('/content/temp1/shopping/'):
#   im=Image.open('/content/temp1/shopping/{}'.format(file))
#   im_mirror=ImageOps.mirror(im)
#   im_mirror.save('/content/temp1/shopping/flip_{}'.format(file))
# print('shopping : ',len(os.listdir('/content/temp1/shopping/')))

# !mkdir temp1/travelling
# unzip('/content/drive/My Drive/Colab Notebooks/datasets/Travelling_Data.zip','/content/temp1')
# for folder in os.listdir('/content/temp1/Travelling_Data/'):
#   folder_path='/content/temp1/Travelling_Data/'+folder+'/'
#   i=0
#   for file in os.listdir(folder_path):
#     if(i>62):
#       break
#     try:
#       file_path=folder_path+file
#       shutil.copy(file_path,'/content/temp1/travelling/{}.png'.format(file))
#       im=Image.open(file_path)
#       im_mirror=ImageOps.mirror(im)
#       i=i+1
#       im_mirror.save('/content/temp1/travelling/flip_{}.png'.format(file))
#     except:
#       pass
# print('travelling : ',len(os.listdir('/content/temp1/travelling/')))

## Setup Input Pipeline

Download the flowers dataset.

In [0]:
_URL = "https://doc-c4-2c-drive-data-export.googleusercontent.com/download/mkqaimh73a7qvdd7v1hq6mliq8gv18o1/9djtb1edsd7ma1u7c6p04ilctav186qg/1588761000000/7136277d-70e2-43dd-bcd5-35170a969db2/114021751865500122868/ADt3v-MCudxwprYaAqPROQM7m5yXVAFymUc7Nb1PfDAIC9Vm7V1HS1KWD2cXhkBQwy677gtuhMhXHzwckJZ6tnPzBIsgsH9VR8aU1lOs3I6prl1E-2X46x4aAmPIUi39o7KgyU1LpCbZ83vDvHXpprC0nfGh9ppdf_JzgKQkY6CGJ6dRTxVpkIUldRGj3ARhuDMbFWrb0RNP_VPeEi-PGhEKTqbvebYdOZBehn_pgHrk9BUiy3PF28eFOL2fW5TyhK_Yq72RhJzM4DIsNR3lHN5Tzfx0OT62ooNtzC3YOqTU4IpvjxgTF9p44oAE8zR7blTVAENulLeN?authuser=0&nonce=0h2c37krj0frq&user=114021751865500122868&hash=6jr6bgfotpdi9ah9kpuj4fku5aebnjtt/"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.zip", 
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

In [0]:
os.listdir(base_dir)

Use `ImageDataGenerator` to rescale the images.

Create the train generator and specify where the train dataset directory, image size, batch size.

Create the validation generator with similar approach as the train generator with the flow_from_directory() method.

In [0]:
IMAGE_SIZE = 128
BATCH_SIZE = 32
base_dir='/content/temp1'
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.1,zoom_range=0.1,rotation_range=0.1,shear_range=0.1,brightness_range=[0.8,1.0],height_shift_range=0.1,width_shift_range=0.1)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

Save the labels in a file which will be downloaded later.

In [0]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [0]:
!cat labels.txt

In [0]:
# shutil.make_archive('running_riding','zip','/content/temp1')
# shutil.copy('/content/running_riding.zip','/content/drive/My Drive/Colab Notebooks/datasets')

## Create the base model from the pre-trained convnets

Create the base model from the **MobileNet V2** model developed at Google, and pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images.

First, pick which intermediate layer of MobileNet V2 will be used for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the `include_top=False` argument, we load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

## Feature extraction
You will freeze the convolutional base created from the previous step and use that as a feature extractor, add a classifier on top of it and train the top-level classifier.

In [0]:
base_model.trainable = False

### Add a classification head

In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10, activation='softmax')
])

### Compile the model

You must compile the model before training it.  Since there are multiple classes, use a categorical cross-entropy loss.

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Train the model

<!-- TODO(markdaoust): delete steps_per_epoch in TensorFlow r1.14/r2.0 -->

In [0]:
epochs = 15

history = model.fit(train_generator, 
                    steps_per_epoch=len(train_generator), 
                    epochs=epochs, 
                    validation_data=val_generator, 
                    validation_steps=len(val_generator))

### Learning curves

Let's take a look at the learning curves of the training and validation accuracy/loss when using the MobileNet V2 base model as a fixed feature extractor. 

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# print (train_generator.class_indices)

# labels = '\n'.join(sorted(train_generator.class_indices.keys()))
# act
# with open('labels.txt', 'w') as f:
#   f.write(labels)
# no=int(input())
# plt.imshow(val_generator[0][0][no].reshape(128,128,3))
# plt.show()
# model.predict(val_generator[0][0][no].reshape(-1,128,128,3))
print(act)
img= cv2.imread('/content/part4.jpg')

img=cv2.resize(img,(128,128))
plt.imshow(img)
plt.show()
img=np.array(img)/255
a=model.predict(img.reshape(-1,128,128,3))
print(a)
act[np.argmax(a)]

In [0]:
# os.remove('temp1/demo.jpg')
dirpath,name=download_file("https://www.vmcdn.ca/f/files/okotokstoday/images/jonny-harris.jpg","demo")
img=cv2.imread(dirpath)
plt.imshow(img)
plt.show()
img=cv2.resize(img,(128,128))
img=np.array(img)/255
a=model.predict(img.reshape(-1,128,128,3))
print(a)
print(act)
print(act[np.argmax(a)])

## Fine tuning
In our feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were **not** updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic features maps to features associated specifically to our dataset.

In [0]:
# os.remove('/content/data/demo3.jpg')
# dirname,name=download_file('https://image.freepik.com/free-photo/phoning-by-work_1098-12754.jpg','demo3')
# img= cv2.imread(dirname)
act=['brushing_teeth','cooking','drinking','gardening','party','phoning','riding_a_bike','riding_a_horse','running','taking_photos']
for folder in os.listdir('test/'):
  path='test/{}/'.format(folder)
  for file in os.listdir(path): 
    img= cv2.imread(path+file)
    img=cv2.resize(img,(128,128))
    plt.imshow(img)
    plt.show()
    img=np.array(img)/255
    a=model.predict(img.reshape(-1,128,128,3))
    print(a)
    print(act[np.argmax(a)])

In [0]:
pred=[]
actual=[]
val=val_generator
n=len(val)
for i in range(n):
  for one in range(32):
    try:
      img=val[i][0][one]
      a=model.predict(img.reshape(-1,128,128,3))
      a=np.argmax(a)
      pred.append(act[a])
      # print(act[a])
      actual.append(act[np.argmax(val[i][1][one])])
      # print(act[np.argmax(val[1][one])])
    except:
      print(one)
      break

In [0]:
print(act)
plt.figure(figsize=(12,8))
sns.heatmap(confusion_matrix(actual,pred),annot=True)
print(classification_report(pred,actual))

### Un-freeze the top layers of the model


All you need to do is unfreeze the `base_model` and set the bottom layers be un-trainable. Then, recompile the model (necessary for these changes to take effect), and resume training.

In [0]:
base_model.trainable = True

In [0]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Compile the model

Compile the model using a much lower training rate.

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

### Continue Train the model

In [0]:
history_fine = model.fit(train_generator, 
                         steps_per_epoch=len(train_generator), 
                         epochs=5, 
                         validation_data=val_generator, 
                         validation_steps=len(val_generator))

## Convert to TFLite

Saved the model using `tf.saved_model.save` and then convert the saved model to a tf lite compatible format.

In [0]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Download the converted model and labels

In [0]:
from google.colab import files

# files.download('model.tflite')
files.download('labels.txt')

Let's take a look at the learning curves of the training and validation accuracy/loss, when fine tuning the last few layers of the MobileNet V2 base model and training the classifier on top of it. The validation loss is much higher than the training loss, so you may get some overfitting.

You may also get some overfitting as the new training set is relatively small and similar to the original MobileNet V2 datasets.


In [0]:
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Summary:

* **Using a pre-trained model for feature extraction**:  When working with a small dataset, it is common to take advantage of features learned by a model trained on a larger dataset in the same domain. This is done by instantiating the pre-trained model and adding a fully-connected classifier on top. The pre-trained model is "frozen" and only the weights of the classifier get updated during training.
In this case, the convolutional base extracted all the features associated with each image and you just trained a classifier that determines the image class given that set of extracted features.

* **Fine-tuning a pre-trained model**: To further improve performance, one might want to repurpose the top-level layers of the pre-trained models to the new dataset via fine-tuning.
In this case, you tuned your weights such that your model learned high-level features specific to the dataset. This technique is usually recommended when the training dataset is large and very similar to the orginial dataset that the pre-trained model was trained on.
